# COMP 484 - Practical Assignment 3

#### Ramraj Chimouriya
#### CE IV/I

## Chapter 6 - Clustering – Finding Related Posts

___

### Converting raw test into a bag of words

Scikit's CounterVectorizer method count words and represent those counts as a vector.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)

Consider the following examples:

In [3]:
content = ["How to format my hard disk",
           "Hard disk format problems"]

X = vectorizer.fit_transform(content)
vectorizer.get_feature_names_out()

array(['disk', 'format', 'hard', 'how', 'my', 'problems', 'to'],
      dtype=object)

In [4]:
print(X.toarray().transpose())

[[1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]


### Counting words

Reading posts from data/toy directory and feeding to CountVectorizer.

In [5]:
from pathlib import Path
TOY_DIR = Path("data/toy")
posts = []
for fn in TOY_DIR.iterdir():
    with open(fn, 'r') as f:
        posts.append(f.read())

In [6]:
vectorizer = CountVectorizer(min_df=1)

In [7]:
X_train = vectorizer.fit_transform(posts)
num_samples, num_features = X_train.shape
print(f"samples: {num_samples} \t features:{num_features}")

samples: 5 	 features:25


In [8]:
print(vectorizer.get_feature_names_out())

['about' 'actually' 'capabilities' 'contains' 'data' 'databases' 'images'
 'imaging' 'interesting' 'is' 'it' 'learning' 'machine' 'most' 'much'
 'not' 'permanently' 'post' 'provide' 'save' 'storage' 'store' 'stuff'
 'this' 'toy']


Now, we can vectorize our new post:

In [9]:
new_post = "Imaging databases"
new_post_vec = vectorizer.transform([new_post])

In [10]:
print(new_post_vec)

  (0, 5)	1
  (0, 7)	1


In [11]:
print(new_post_vec.toarray())

[[0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


For the similarity measurement (the naive one), we calculate the Euclidean distance between the count vectors of the new post and all the old posts.

In [12]:
import scipy
def dist_raw(v1, v2):
    delta = v1-v2
    return scipy.linalg.norm(delta.toarray())

Function that takes the current dataset and the new post in vectorized form as well as a distance function and prints out an analysis of how well the distance function works:

In [13]:
def best_post(X, new_vec, dist_func):
    best_doc = None
    best_dist = float('inf')    #infinite value as a starting point
    best_i = None
    for i, post in enumerate(posts):
        if post == new_post:
            continue
        post_vec = X.getrow(i)
        d = dist_func(post_vec, new_vec)
        print(f"=== Post {i} with dist={round(d,2)}: \n'{post}'")
        if d < best_dist:
            best_dist = d
            best_i = i
    print(f"===> Best post is {best_i} with dist={round(best_dist,2)}")

In [14]:
best_post(X_train, new_post_vec, dist_raw)

=== Post 0 with dist=1.41: 
'Imaging databases store data.'
=== Post 1 with dist=2.0: 
'Most imaging databases save images permanently.
'
=== Post 2 with dist=1.73: 
'Imaging databases provide storage capabilities.'
=== Post 3 with dist=4.0: 
'This is a toy post about machine learning. Actually, it contains not much interesting stuff.'
=== Post 4 with dist=5.1: 
'Imaging databases store data. Imaging databases store data. Imaging databases store data.'
===> Best post is 0 with dist=1.41


### Normalizing word count vectors`

In [15]:
def dist_norm(v1, v2):
    v1_normalized = v1 / scipy.linalg.norm(v1.toarray())
    v2_normalized = v2 / scipy.linalg.norm(v2.toarray())
    delta = v1_normalized - v2_normalized
    return scipy.linalg.norm(delta.toarray())

In [16]:
best_post(X_train, new_post_vec, dist_norm)

=== Post 0 with dist=0.77: 
'Imaging databases store data.'
=== Post 1 with dist=0.92: 
'Most imaging databases save images permanently.
'
=== Post 2 with dist=0.86: 
'Imaging databases provide storage capabilities.'
=== Post 3 with dist=1.41: 
'This is a toy post about machine learning. Actually, it contains not much interesting stuff.'
=== Post 4 with dist=0.77: 
'Imaging databases store data. Imaging databases store data. Imaging databases store data.'
===> Best post is 0 with dist=0.77


### Removing less important words

In [17]:
vect_engl = CountVectorizer(min_df=1, stop_words='english')

In [18]:
sorted(vect_engl.get_stop_words())[0:20]

['a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst']

In [20]:
X_train_engl = vect_engl.fit_transform(posts)
num_samples_engl, num_features_engl = X_train_engl.shape
print(f"#samples: {num_samples_engl}, #features: {num_features_engl}")
print(vect_engl.get_feature_names_out())

#samples: 5, #features: 18
['actually' 'capabilities' 'contains' 'data' 'databases' 'images'
 'imaging' 'interesting' 'learning' 'machine' 'permanently' 'post'
 'provide' 'save' 'storage' 'store' 'stuff' 'toy']


In [23]:
new_post_vec_engl = vect_engl.transform([new_post]) 

best_post(X_train_engl, new_post_vec_engl, dist_norm)

=== Post 0 with dist=0.77: 
'Imaging databases store data.'
=== Post 1 with dist=0.86: 
'Most imaging databases save images permanently.
'
=== Post 2 with dist=0.86: 
'Imaging databases provide storage capabilities.'
=== Post 3 with dist=1.41: 
'This is a toy post about machine learning. Actually, it contains not much interesting stuff.'
=== Post 4 with dist=0.77: 
'Imaging databases store data. Imaging databases store data. Imaging databases store data.'
===> Best post is 0 with dist=0.77


### Stemming

In [24]:
import nltk.stem

In [25]:
s = nltk.stem.SnowballStemmer('english')

In [26]:
s.stem("graphics")

'graphic'

### Extending the vectorizer with NLTK's stemmer

In [27]:
english_stemmer = nltk.stem.SnowballStemmer('english')

In [38]:
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

In [39]:
vect_engl_stem = StemmedCountVectorizer(min_df=1, stop_words='english')
print(vect_engl_stem)

StemmedCountVectorizer(stop_words='english')


In [41]:
X_train_engl_stem = vect_engl_stem.fit_transform(posts)
num_samples_engl_stem, num_features_engl_stem = X_train_engl_stem.shape
print(f"#samples: {num_samples_engl_stem}, #features: {num_features_engl_stem}")
print(vect_engl_stem.get_feature_names())

#samples: 5, #features: 17
['actual', 'capabl', 'contain', 'data', 'databas', 'imag', 'interest', 'learn', 'machin', 'perman', 'post', 'provid', 'save', 'storag', 'store', 'stuff', 'toy']
